In [75]:
# TFlearn helper to setup neural networks
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences

# Tools to scrape the web
import re
import urllib
import requests
from collections import Counter
from bs4 import BeautifulSoup, SoupStrainer

# Data wrangling modules
import numpy as np
import pandas as pd

In [2]:
# Read ign data

ignd = pd.read_csv('ign.csv', index_col=0)

In [3]:
# Shape of the data

ignd.shape

(18625, 10)

In [68]:
# Function to scrape the IGN website for verdicts

# class VerdictScraper:
#     def __init__(self):
#         self.session = requests.Session()

def get_review_verdict(game_url):
    url = 'http://ca.ign.com%s' % (game_url)
    # response = self.session.get(url, headers={'User-agent': 'Mozilla/5.0'})
    response = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})

    # parse the search page using SoupStrainer and lxml
    soup = BeautifulSoup(response.content, 'lxml')

    # Get the review link
    rev_tag = soup.find(name="a", class_="reviewLink", string=re.compile('Review'))
    try:
        rev_url = rev_tag.attrs['href']

        # Extract the verdict of the review
        response = requests.get(rev_url, headers={'User-agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(response.content, 'lxml')
        verdict_tag = soup.find(name='div', class_='articleSubHeader', string=re.compile('The Verdict'))

        verdict = verdict_tag.next_sibling.next_sibling.next_sibling
        verdict_str = ''.join([s for s in verdict.strings])
    except:
        verdict_str = None

    return verdict_str

In [70]:
# Fill in the reviews (this takes a while)

ignd['Reviews'] = ignd['url'].map(lambda x: get_review_verdict(x))

# scraper = VerdictScraper()

In [72]:
ignd.to_csv('ign_reviews.csv')

In [96]:
# Loop and create bag of word

bw = Counter()
mask = ignd['Reviews'].isnull() == False

char_to_sub = '|'.join(['\(', '\)', ',', '\.', '\?', '!', ])
pat = re.compile(char_to_sub) 

for rev in ignd.loc[mask, 'Reviews']:
    rev = rev.lower()
    rev = pat.sub('', rev)
    words = rev.split()
    for w in words:
        bw[w] += 1

In [98]:
bw.most_common(10000)

[('the', 53618),
 ('a', 32776),
 ('and', 27817),
 ('of', 25490),
 ('to', 24710),
 ('is', 17259),
 ('game', 13887),
 ('it', 13523),
 ('that', 12050),
 ('for', 11326),
 ('in', 11305),
 ('but', 10708),
 ('you', 9064),
 ('this', 8521),
 ('with', 7796),
 ('on', 7476),
 ('as', 6926),
 ('i', 6364),
 ('be', 5584),
 ('if', 5455),
 ("it's", 5162),
 ('its', 4971),
 ('are', 4758),
 ('more', 4584),
 ('just', 4273),
 ('an', 4225),
 ('have', 4165),
 ('not', 4123),
 ('games', 3655),
 ('or', 3637),
 ('one', 3529),
 ('like', 3494),
 ('fun', 3333),
 ('at', 3283),
 ('your', 3266),
 ('all', 3040),
 ('some', 3015),
 ('from', 2971),
 ('out', 2887),
 ('up', 2872),
 ('will', 2804),
 ('so', 2674),
 ('there', 2670),
 ('can', 2607),
 ('even', 2588),
 ('good', 2580),
 ('has', 2564),
 ('than', 2519),
 ('still', 2461),
 ('--', 2405),
 ('play', 2353),
 ('great', 2300),
 ('was', 2283),
 ('time', 2267),
 ('get', 2171),
 ('much', 2147),
 ('really', 2108),
 ('new', 2078),
 ('gameplay', 2074),
 ('by', 1994),
 ('what', 191